In [ ]:
# Устанавливаем необходимые библиотеки:
%pip install transformers datasets evaluate tensorflow
# - transformers: библиотека для работы с моделями трансформеров
# - datasets: библиотека для удобной работы с наборами данных
# - evaluate: инструменты для оценки моделей
# - tensorflow: фреймворк для машинного обучения и глубокого обучения

In [ ]:
from huggingface_hub import login  # Импорт функции для авторизации на Hugging Face Hub
from datasets import load_dataset  # Импорт функции для загрузки наборов данных
import tensorflow as tf  # Импорт TensorFlow для работы с моделями и вычислениями
from transformers import AutoTokenizer  # Импорт токенизатора для обработки текста
from transformers import DefaultDataCollator  # Импорт инструмента для подготовки данных для модели
from transformers import create_optimizer  # Импорт функции для создания оптимизатора
from transformers import TFAutoModelForQuestionAnswering  # Импорт модели для задачи вопросов и ответов
from transformers.keras_callbacks import PushToHubCallback  # Импорт callback для отправки модели на Hugging Face Hub
from transformers import pipeline  # Импорт инструмента для создания готовых pipelines

In [ ]:
HF_TOKEN = ""  # Токен для авторизации на Hugging Face Hub (нужно вставить ваш токен)
login(HF_TOKEN, add_to_git_credential=True)  # Авторизация на Hugging Face Hub с использованием токена
# Параметр `add_to_git_credential=True` добавляет токен в git-credential для удобства работы с репозиториями

In [ ]:
# Загружаем набор данных SQuAD (Stanford Question Answering Dataset)
# SQuAD содержит пары вопросов и ответов на основе текста
# Используем только первые 5000 примеров для ускорения обработки
dataset = load_dataset("squad", split="train[:5000]")

# Разделяем данные на обучающую и тестовую выборки
# test_size=0.2 означает, что 20% данных будут использоваться для тестирования, а 80% — для обучения
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
# Загружаем предобученный токенизатор для модели DistilBERT
# Модель "distilbert-base-uncased" — это уменьшенная версия BERT без учёта регистра символов
# Токенизатор преобразует текст в числовые токены, которые могут быть обработаны моделью
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
@tf.autograph.experimental.do_not_convert  # Отключаем автоматическое преобразование TensorFlow Autograph для этой функции
def process_data(examples):
    # Очищаем вопросы от лишних пробелов
    questions = [q.strip() for q in examples["question"]]
    
    # Токенизируем вопросы и контекст
    # max_length=384: ограничиваем длину входных данных до 384 токенов
    # truncation="only_second": обрезаем только контекст, если он превышает максимальную длину
    # return_offsets_mapping=True: возвращаем информацию о позициях символов для каждого токена
    # padding="max_length": дополняем последовательности до максимальной длины
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Извлекаем информацию о позициях символов для каждого токена
    offset_mapping = inputs.pop("offset_mapping")
    
    # Получаем ответы из данных
    answers = examples["answers"]
    
    # Списки для хранения начальных и конечных позиций ответов
    start_positions = []
    end_positions = []

    # Обрабатываем каждый пример
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        # Определяем начальную и конечную позиции ответа в тексте
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        
        # Получаем идентификаторы последовательностей (0 — вопрос, 1 — контекст)
        sequence_ids = inputs.sequence_ids(i)

        # Находим начало контекста
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        
        # Находим конец контекста
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Проверяем, находится ли ответ в пределах контекста
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            # Если ответ вне контекста, указываем позиции 0
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Находим начальную позицию ответа в токенах
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            # Находим конечную позицию ответа в токенах
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    # Добавляем начальные и конечные позиции в inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    # Возвращаем обработанные данные
    return inputs

In [ ]:
# Применяем функцию `process_data` к набору данных
# batched=True: обработка данных происходит пакетами для ускорения
# remove_columns=dataset["train"].column_names: удаляем исходные столбцы, так как они больше не нужны
tokenized_dataset = dataset.map(process_data, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
# Создаем объект для подготовки батчей данных
# return_tensors="tf": данные будут возвращаться в формате TensorFlow
data_collator = DefaultDataCollator(return_tensors="tf")

# Определяем размер батча и количество эпох
batch_size = 16  # Количество примеров в одном батче
num_epochs = 2  # Количество эпох для обучения

# Вычисляем общее количество шагов обучения
# len(tokenized_dataset["train"]) — количество примеров в обучающей выборке
# // batch_size — количество батчей за одну эпоху
# * num_epochs — общее количество шагов для всех эпох
total_train_steps = (len(tokenized_dataset["train"]) // batch_size) * num_epochs

# Создаем оптимизатор и расписание learning rate
# init_lr=2e-5: начальный learning rate
# num_warmup_steps=0: количество шагов "разогрева" (в данном случае отсутствует)
# num_train_steps=total_train_steps: общее количество шагов обучения
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
# Загружаем предобученную модель для задачи вопросов и ответов (Question Answering)
# Модель основана на архитектуре DistilBERT (уменьшенная версия BERT)
# "distilbert-base-uncased" — это версия модели, обученная на текстах без учёта регистра
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
# Подготавливаем TensorFlow Dataset для обучающих данных
# tokenized_dataset["train"]: токенизированные обучающие данные
# shuffle=True: перемешивание данных для улучшения обучения
# batch_size=16: размер батча
# collate_fn=data_collator: функция для создания батчей (используется DefaultDataCollator)
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

# Подготавливаем TensorFlow Dataset для валидационных данных
# tokenized_dataset["test"]: токенизированные тестовые данные
# shuffle=False: данные не перемешиваются (для корректной оценки)
# batch_size=16: размер батча
# collate_fn=data_collator: функция для создания батчей
tf_validation_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
# Создаем callback для отправки модели на Hugging Face Hub после обучения
# output_dir="my_qa_model": директория, куда будет сохранена модель
# tokenizer=tokenizer: токенизатор, который будет загружен вместе с моделью
callback = PushToHubCallback(
    output_dir="my_qa_model",
    tokenizer=tokenizer,
)

In [ ]:
# Компилируем модель с выбранным оптимизатором
# optimizer=optimizer: используем созданный ранее оптимизатор
model.compile(optimizer=optimizer)

# Обучаем модель
# x=tf_train_set: обучающие данные
# validation_data=tf_validation_set: данные для валидации
# epochs=5: количество эпох обучения
# callbacks=[callback]: используем callback для отправки модели на Hugging Face Hub
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=[callback])

In [ ]:
# Задаем вопрос и контекст
question = "When was the first iPhone released?"
context = "The first iPhone was released on June 29, 2007. It was a revolutionary product that changed the smartphone industry."

# Загружаем токенизатор из сохраненной модели
tokenizer = AutoTokenizer.from_pretrained("my_qa_model")

# Токенизируем вопрос и контекст
# return_tensors="tf": возвращаем данные в формате TensorFlow
inputs = tokenizer(question, context, return_tensors="tf")

# Загружаем модель для задачи вопросов и ответов
model = TFAutoModelForQuestionAnswering.from_pretrained("my_qa_model")

# Получаем выходные данные модели (логиты для начальной и конечной позиции ответа)
outputs = model(**inputs)

# Находим индекс начала ответа (токен с максимальным значением start_logits)
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])

# Находим индекс конца ответа (токен с максимальным значением end_logits)
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

# Извлекаем токены ответа из входных данных
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

# Декодируем токены обратно в текст
tokenizer.decode(predict_answer_tokens)

In [ ]:
question = "Who wrote the novel 'To Kill a Mockingbird'?"
context = "The novel 'To Kill a Mockingbird' was written by Harper Lee and published in 1960."

tokenizer = AutoTokenizer.from_pretrained("my_qa_model")
inputs = tokenizer(question, context, return_tensors="tf")

model = TFAutoModelForQuestionAnswering.from_pretrained("my_qa_model")
outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)


Some layers from the model checkpoint at my_qa_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at my_qa_model and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'harper lee'

In [ ]:
question = "What is the capital of France?"
context = "The capital of France is Paris. It is known for its art, culture, cuisine, and fashion."

tokenizer = AutoTokenizer.from_pretrained("my_qa_model")
inputs = tokenizer(question, context, return_tensors="tf")

model = TFAutoModelForQuestionAnswering.from_pretrained("my_qa_model")
outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)


Some layers from the model checkpoint at my_qa_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at my_qa_model and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'paris'